### 计算PNL（一）、 计算手续费收入

1. 选定一个池子，获取池子的基本信息 ✅
2. 选定一个头寸和时间范围之后，获得这段时间内池子里的所有swap记录 ✅
3. 获取池子流动性的时间序列数据 ✅
4. 结合3和4计算出选定的头寸在这段时间内的手续费收入 

In [358]:

import sys 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.getcwd()))
# 这是为了自动重载自定义包的内容（python好不方便呀）
import importlib
import poolData.config
import poolData.swapData as swapData

importlib.reload(poolData.config)
importlib.reload(swapData)

<module 'poolData.swapData' from '/Users/bcfh/Documents/GitHub/learn_uniswapv3/poolData/swapData.py'>

In [29]:
from datetime import datetime
begin = datetime(2022, 12, 31)
end = datetime(2023, 1, 1)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"

### 1. 获取池子的基本信息

https://etherscan.io/address/0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#readContract

手续费率：0.05%

protocol fee: 0.05%

### 2. 获取所有swpa记录

In [30]:
data = swapData.query_swaps(int(begin.timestamp()), int(end.timestamp()), pool_id)
data

,id,amount0,amount1,timestamp,amountUSD,sqrtPriceX96
0,0x0002088b2f326f0114b41b8a89d7f15f12d293328a6a...,4266.117177,-3.560090738149703786,1672447895,4264.941859937119356472465091232485,2289294668390795750019773769053102
1,0x0013b93196471c4dcb508221cd6a0bbda1e6bdc654b3...,1000,-0.836888527815654406,1672421291,999.7670959814923754131727434898885,2292565998538329889197212915778332
2,0x0013fce47ecd63a38700ff544d31d9cbf95ccdfccb02...,-5011.751045,4.2,1672422803,5014.00285661645623131938387967046,2292987089929314170924596543568948
3,0x00151559f96c8ad818a4847f123c1f14fa4974f84b09...,2824.243528,-2.355025735643548311,1672438979,2822.308348680553578867706064338535,2288414245600305261854951574027895
4,0x001728914057df211c141e2183ba0409d30c1c099ff4...,99.563794,-0.082988052797955076,1672436315,99.47429731086213279113076465711465,2287943199586960584738206920087911
...,...,...,...,...,...,...
95,0x04a2fd9ddae5fde5ff1a61501be43c105d0300d2e7ff...,239.13294,-0.2,1672431047,239.0001518960397064507089610970602,2291835523874819000217951015443670
96,0x04a396a97abe63a1306211cf0888b83ad0441f5e2876...,-1194.390617,1,1672472027,1194.775439063682457110202425296138,2291912134395744923818529532811145
97,0x04a7c7c32ca5d422f1ea6ba40ac4b8f13c72962f1df6...,-115,0.09636875774880145,1672467539,115.0881251669916150099873981421359,2292926901264833166510303843797916
98,0x04ac502165376e6fadd2d6c37b37802743063f091027...,-777.732055,0.65173269239186,1672467599,778.329010541821646581812490672832,2292930270926876268162122247869437


### 3. 获取池子流动性的时间序列数据

In [31]:
data = swapData.query_liquidity(int(begin.timestamp()), int(end.timestamp()), pool_id)
data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1672419600,43383970859571409238,2292741488478236193954710948079555,1194.122792146251869705949121256262,0.0008374348153950348453709271537854136,205468,2186941498930609995212528368989333,1031090019324772402022075697810397089878707,333609426.6969159360891818260545127,0,0,0,0,225,1194.511809050204970771435064936336,1195.135863873698740398661399390146,1193.804391237179988357396971990688,1194.122792146251869705949121256262
1,1672423200,41908297001142436164,2289914226033824321765933548671387,1197.073280553630480706658776824827,0.000835370746507275888776747615886998,205444,2186952021398667747348751967478818,1031092959063940135596058161042444273527573,333488757.0211926141408956767590597,0,0,0,0,260,1194.123444963926810270341103316625,1197.073280553630480706658776824827,1194.080786262999696020970218231062,1197.073280553630480706658776824827
2,1672426800,43259411757668858895,2292095502081045216146554614279866,1194.795971584433226686676411440843,0.0008369629826202778712332352962748905,205463,2186954913044984967466704176819519,1031099918456639174812780045987174854968176,333637095.322414727186961463831933,0,0,0,0,289,1197.073280553630480706658776824827,1197.07382117549381742907326713768,1194.750385533700741374110953471534,1194.795971584433226686676411440843
3,1672430400,41853986357057955972,2290601372199165019827373276558852,1196.355180105557782621625188956052,0.0008358721695941226979286513996652267,205450,2186962943960877559886609212538257,1031103373243662410724723254463120782143002,333906894.4645702736186860023230338,0,0,0,0,263,1194.796131311705043167380267771029,1196.355180105557782621625188956052,1194.693649299187396892654724716834,1196.355180105557782621625188956052
4,1672434000,59109260845469698637,2288829923181971195179725662338341,1198.207744422359329581249063675693,0.0008345798169432523948316314255772096,205434,2186981817612323087894790173617207,1031115295566281377942677265799122428836270,334440380.7305554361225720897040363,0,0,0,0,338,1196.394407853511805846627602601061,1200.916144596237448207735051238417,1196.394407853511805846627602601061,1198.207744422359329581249063675693
5,1672437600,42635286778298209243,2289840993430537869827068272854702,1197.149850232257000874432172955406,0.0008353173162123285613392883684579638,205443,2186985466783348684010015616485416,1031121171227709197569329131978729692462092,334159110.5215818807642015737513923,0,0,0,0,284,1198.206832687504706963694948656337,1198.788882736739192695900153120176,1196.572152892231302126497021777375,1197.149850232257000874432172955406
6,1672441200,42266969974656527701,2288242077353913887971525628274079,1198.823458909347496249755693340487,0.0008341511776135654455497975422011697,205429,2186997074931896828965175654912667,1031126766030148915041621606252563195946542,334188221.1952228731733906900809821,0,0,0,0,270,1197.82449644040087986756220591501,1199.244261599893039949720652423436,1197.824418695737796824551141160768,1198.823458909347496249755693340487
7,1672444800,42375952704822449015,2289495616475923548060168410771042,1197.511064425397845989152045989084,0.0008350653532206237699360214312313554,205440,2187003010215967611186322600569947,1031134413949733663273603963765900329834324,334113410.6901975287201155154576156,0,0,0,0,276,1198.825324382086128819092982674148,1198.842790767155624227608013196213,1197.056358947755097837177674114186,1197.511064425397845989152045989084
8,1672448400,41797181380235020195,2290877490356118393288567624905111,1196.066805470759543825298400099456,0.0008360737004204462694587878968945214,205452,2187005870139216598257742201908262,1031139772075510180972976441088333388304521,334115162.1019867095005736493665301,0,0,0,0,246,1197.510920373478583418711445267929,1198.119081909486525458850696326448,1196.065922163147825920797406752285,1196.066805470759543825298400099456
9,16724520

### 4. 计算头寸的手续费分成收入

参数解释：
1. 不要忘记protocol fee(需要从池子合约中获取)

    swap的手续费不是所有都给流动性提供者的，一部分会给协议，最终由dao收取。

2. feeGrowthGlobal0X128 和 feeGrowthGlobal1X128

    feeGrowthGlobal0X128:  The all-time global fee growth, per unit of liquidity, in token0

    feeGrowthGlobal1X128:  The all-time global fee growth, per unit of liquidity, in token1

    这是池子合约中的两个全局状态变量，池子每一次swap都会自动更新这两个值，表示的是整个池子里手续费的累计增长率，X128表示这个值是乘$ 2 ^ {128} $之后的值，所以在实际使用的时候需要把这个值除以$ 2 ^ {128} $.

    feeGrowthGlobal的计算方法：

    $$ feeGrowthGlobal0X128 = \frac{accumulative fees \times 2 ^ {128}}{total liquidity} $$

3. feeGrowthOutside0X128 和 feeGrowthOutside1X128

    这两个变量的定义和上面的global差不多，差别是feeGrowthOutside0X128是记录一个tick外所有的手续费增长率

    知道了feeGrowthOutside0X128和feeGrowthGlobal0X128的值，就可以计算出每次swap之后该position能获得的手续费是多少。
    
考虑两种情况：

a. swap是在你的tick range内发生的

$$ feetoken0 = \frac {(feeGrowthGlobal0X128 - feeGrowthOutside0X128_lower - feeGrowthOutside0X128_upper - feeGrowthInside0LastX128) \times liquidity}{ 2 ^ {128} \times 10 ^ {decimal}} $$

b. swap发生后你的tick out of range了

    i. current ticker > tick upper
        $$ token0fee = \frac {(feeGrowthOutside0X128_upper - feeGrowthOutside0X128_lower - feeGrowthInside0LastX128) \ times liquidity } { 2 ^ {128}  \times 10 ^ {decimal} } $$
    
    ii. current ticker < tick upper
        $$ token0fee = \frac {(feeGrowthOutside0X128_lower - feeGrowthOutside0X128_upper - feeGrowthInside0LastX128) \ times liquidity } { 2 ^ {128}  \times 10 ^ {decimal} } $$

***注意***

在the graph中能拿到的最细粒度数据是小时级别的，用以上的计算方法存在一个误差点：

利用小时级别的feeGrowth数据来计算手续费收入，忽视了一个小时内池子流动性发生的变化。如果在这一个小时内，有流动性提供者推出或者新的流动性提供者加入，那么feeGrowth数据其实是会发生变化的。

但这个误差是可以容忍的，所以我们假设一个小时内池子的流动性不发生变化。

4. fee

在v2中，收取的手续费会自动转变成liquidity，随着池子内swap不断发生，你的liquidity也会逐渐积累。

但在v3中，收取的手续费不再自动转成liquidity，背后的原因是：virtual reserve的存在导致收取的手续费无法直接按比例加入池子成为流动性。

fee的思考：

v3的特点就是允许你在某几个tick内提供流动性，需要一个比较好的方式来存储以tick为单位的手续费，所以v3记录手续费的方式是用全局变量记录整个池子内的手续费增长情况 + 每个tick的手续费增长情况，和直接记录手续费大小相比这是一个优化后的数据结构设计。这一部分的内容在白皮书的6.3左右：

swap的流程：
![swap-control](formulas/swap-control.png)

对于每一个tick，合约中都会存储以下这些变量：

1. liquidityNet：

    the total amount of liquidity that should be kicked in or out when the tick is crossed. 
    <!-- $$ \Delta L = \frac {} $$ -->
2. liquidityGross
3. feeGrowthOutside0X128
4. feeGrowthOutside1X128
5. secondsOutside
6. tickCumulativeOutside
7. secondsPerLiquidityOutsideX128

另外这个文档中也有很详细的解释：
https://uniswapv3book.com/docs/milestone_5/swap-fees/#:~:text=Fees%20are%20paid%20by%20users,amount%20divided%20by%20pool's%20liquidity).

To make fees accounting simpler, Uniswap V3 tracks the global fees generated by 1 unit of liquidity. Price range fees are then calculated based on the global ones: fees accumulated outside of a price range are subtracted from the global fees. Fees accumulated outside of a price range are tracked when a tick is crossed (and ticks are crossed when swaps move the price; fees are collected during swaps). With this approach, we don’t need to update fees accumulated by each position on every swap–this allows to save a lot of gas and make interaction with pools cheaper.

### 5. 获取tick的数据

获取一个池子内所有的被initialized的ticks

In [32]:
data = swapData.query_ticks("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8")
data

,tickIdx,liquidityGross,liquidityNet,price0,price1,volumeToken0,volumeToken1,volumeUSD,untrackedVolumeUSD,feesUSD,collectedFeesToken0,collectedFeesToken1,collectedFeesUSD,createdAtTimestamp,liquidityProviderCount,feeGrowthOutside0X128,feeGrowthOutside1X128
0,0,0,0,1,1,0,0,0,0,0,0,0,0,1622488494,0,0,0
1,105960,0,0,39953.44719942822385402034558214009,0.00002502912940173810732272813416956387,0,0,0,0,0,0,0,0,1623967517,0,0,0
2,-1080,955255939572502,955255939572502,0.8976324433094195884383843668365469,1.114041729945910239807309095780991,0,0,0,0,0,0,0,0,1620937986,0,119690017018605194611917554219251,30325611353621108165329238915020275919206
3,115140,0,0,100049.8997864926770132351218727111,0.000009995012510097544967562707071231679,0,0,0,0,0,0,0,0,1622536811,0,0,0
4,120660,0,0,173754.1660171966344141422511922068,0.000005755257689194220169430331041597795,0,0,0,0,0,0,0,0,1623419807,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,173400,0,0,33907034.48633657555701262300435812,0.00000002949240519405987126391149071657875,0,0,0,0,0,0,0,0,1620593461,0,0,0
96,173700,16976441980382,16976441980382,34939605.01341065627733928620852156,0.00000002862081582250789831846447283671908,0,0,0,0,0,0,0,0,1621115540,0,143926601842711884409296698321955,36306745908371350016743482107305045565311
97,173820,0,0,35361384.80257631179772826203761658,0.00000002827943547977632812446748555012594,0,0,0,0,0,0,0,0,1623664594,0,0,0
98,-1740,0,0,0.8403042077859023045607653103039314,1.190045213072152005202431109200761,0,0,0,0,0,0,0,0,1659200074,0,0,0


## 关于回测框架

计算头寸的收益最难的就是手续费的部分了，但是如果有position feeGrowthOutside1X128的时间序列数据，结合池子的feeGrowthGlobal0X128对时间序列数据，可以直接计算出一个头寸的手续费收入。

不过为了分析不同策略的pnl（如果不自己写一个的话，只能用现实中真实发生过的头寸进行分析，没有办法做一些策略的设计），我们需要一个回测框架，支持模拟各种头寸的设置，假设头寸不影响池子的深度和价格的情况下分析头寸的pnl。

那么如何计算手续费呢？

两种方法：

### 方法一、
1. 获取池子内所有的swap数据，筛选出在你的头寸之内的所有swap

2. 计算该swap会给你的头寸带来多少手续费收入

### 方法二、
用合约中记录的状态变量：global和outside fee growth

但这个方法只能研究现有的头寸，除非：

**但是我突然有个大胆的想法🙋🏽**

uniswap中的池子不同于传统的金融市场，传统金融市场中做回测是很简单的。比如你想做一个股票的回测框架，你可以不用考虑你的交易对市场价格产生的影响，因为上市的股票往往有着足够的深度和流动性。但是在uniswap中，许多池子都非常非常小，你的头寸很可能会大幅度影响币对的价格。

如果只研究池子深度足够的池子里什么策略好，那上面所说的回测框架是足够了的，但是研究出来的结果对于深度小的池子肯定是不适用的。

能不能自己部署一个uniswap合约，交易数据可以用uniswap上的实际交易数据或者用中心化交易所的交易数据，把这些数据放到自己部署的合约上重放。

· 选一个很深的池子
· 选很小的头寸（这样的话交易gas fee会让pnl为负，也不太好
    ·消除手续费的影响？用在池子内提供mock的token0和token1作为策略的base line？参加：https://pub.tik.ee.ethz.ch/students/2021-HS/BA-2021-21.pdf
    

### 计算手续费分成收入（方法一）

In [33]:
# 先拿一个池子，随便选一个头寸，以便拿计算出来的手续费和网站上的真实手续费比较
data = swapData.query_positions(pool_id, 16859000, 10, "owner")
data

,id,owner,liquidity,depositedToken0,depositedToken1,withdrawnToken0,withdrawnToken1,collectedFeesToken0,collectedFeesToken1,feeGrowthInside0LastX128,feeGrowthInside1LastX128,token0.id,token1.id,tickLower.id,tickUpper.id,transaction.id
0,414897,0x0000000000000000000000000000000000000000,0,8470.656242,1.579300632349748811,9646.002942,0.880907147308836156,0,0,254388150839429989634847833044971,130416506739465878095744363051182666646616,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201600,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#205200,0x55fdd6446251935db1759e2590c374df7a1a44f47976...
1,375094,0x009bbce7a5beae91bc62b44a9ea8ece72420269b,7477222542203439,37122.755264,74.999999999814401363,0,0,0,0,1371319483195357573934983808672903,826385230747468827303810150304635815232023,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#196600,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#208300,0x8fe8da9aa80c22c9a1db08007355607a796ff53ebfb6...
2,465030,0x011839c2298cc26fac6e3971f82aac74ffddea04,0,28976.370507,16.099999999999999986,0,32.490586776097955578,0,0,12920116092859507351442078917099,7360016429594502254918354885069648765303,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201520,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201570,0xaa811dc34060a4e51ec8cb07c626ae4060790fe43aa2...
3,472322,0x011839c2298cc26fac6e3971f82aac74ffddea04,0,24602.143798,2.3905973468482894,28893.940791,0,0,0,24288409362466326980506548861723,13724691144674419161764315747083110152806,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201380,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201530,0x83117aec2082f49e4e1df7a310668a4157a56226b3b9...
4,473062,0x011839c2298cc26fac6e3971f82aac74ffddea04,0,0,16.499999999999999962,0,16.499999999999999961,0,0,1157920892373161954235709850086879078532699840...,1157920892373161954235709850086879073583582457...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201440,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201500,0x107cfe1b1f0104eeffb6cfe5e054fdc2ba557a36a93d...
5,473082,0x011839c2298cc26fac6e3971f82aac74ffddea04,230398931115700438,0,16.499999999999999999,0,0,0,0,1157920892373161954235709850086879078532699840...,1157920892373161954235709850086879073847946717...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201590,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201650,0x39e2935fecd0d0028ca99f2125f2150adb0c36ab298c...
6,469725,0x01529b11392c99cecba4269774b4ceed104b0560,23670320895,0.999618,0.000560498228415169,0,0,0,0,1689766018666622878292657058826068,874179710640166955594482610152259793340903,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#-88...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#887270,0xd6a14cb119798f4a04b9eca602d879931f689cdbec81...
7,464686,0x01b2e49f1f3fbf81db90fc9718672900c12dd249,0,100000,0,99999.999999,0,0,0,263272921443906535322337235723388,167056794808830174104230886640858403101721,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#202050,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#203530,0x568d4d81899d65b3e424e559282a66cfb4755ff19314...
8,467781,0x01b2e49f1f3fbf81db90fc9718672900c12dd249,0,631732.85845,0,631732.858448,0,0,0,493386780164924682514398821643573,360615910827483118564367033307274734108174,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#201760,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#203070,0x035dedfb0920e

In [34]:
# 选定池子,头寸,时间：
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
position_id = 469767

'''
注意1:  poolid在uniswap中是每个头寸的唯一标识:
positionId = keccak256(abi.encodePacked(
    bytes32(poolAddress),
    bytes32(liquidityProvider),
    int24(lowerTick),
    int24(upperTick),
    uint24(fee)
))

注意2: 算完之后可以参考这个看算的对不对：
https://revert.finance/#/uniswap-position/mainnet/469767
'''

data = swapData.query_position(position_id)
data


,id,owner,liquidity,depositedToken0,depositedToken1,withdrawnToken0,withdrawnToken1,collectedFeesToken0,collectedFeesToken1,feeGrowthInside0LastX128,feeGrowthInside1LastX128,token0.id,token1.id,tickLower.id,tickUpper.id,transaction.id
0,469767,0x56209a886c507f8725c822b2b1f77ef4c9dc853c,286206700622530,1466.04493,0.865982013077301422,0,0,0,0,907373610974648595634682261671971,578710287513428657835512647395860822798555,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#198760,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640#204080,0xe2e2041d2b09d5d7ae4f999951775f7ad527a64f3ad1...


### 先简单看一下这个头寸的基本信息：
1. 池子: USDC/WETH
    token0: usdc
    token1: weth
    价格 p = y/x = p_weth / p_usdc
2. usdc: 1466.04493
   eth: 0.865982013077301422
3. 创建时间: March 19, 2023 at 2:17 PM
4. 价格区间：
    需要根据ticker计算一下

$$p(i) = 1.0001^i$$


另外在token合约中存在decimal，usdc的decimal是6，weth的decimal是18。而uniswap在计算的时候是不管decimal的，所以他的价格实际上是：amount_usdc * （10 ** -6）/ amount_weth * (10 ** -18)，整理得到：

$$ p_real = p * (10 ** -12)

In [55]:
p_lower = (1.0001 ** 198760) * (10 ** -12)
p_upper = (1.0001 ** 204080) * (10 ** -12)
print("===price(y/x) the usdc price in terms of weth===")
print("lower price is:", p_lower)
print("higher price is:", p_upper)
p_lower = 1 / ((1.0001 ** 198760) * (10 ** -12))
p_upper = 1 / ((1.0001 ** 204080) * (10 ** -12))
print("===It's easier to read weth price in terms of usdc, so we might inverse the numbers: price(x/y) the weth price in terms of usdc===")
print("lower price is:", p_lower)
print("higher price is:", p_upper)

===price(y/x) the usdc price in terms of weth===
lower price is: 0.00042815946513519496
higher price is: 0.0007288508458564415
===It's easier to read weth price in terms of usdc, so we might inverse the numbers: price(x/y) the weth price in terms of usdc===
lower price is: 2335.5784034442436
higher price is: 1372.0228297532435


In [250]:
# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()
# 获取该时段内池子里所有的swap
swaps = swapData.query_swaps(int(begin.timestamp()), int(end.timestamp()), pool_id)

In [254]:
swaps.shape[0]

100

### 参数解释
首先要理解swap表中的参数。 首先这个表是通过监听uniswap合约中的swap事件得到的，关于swap事件：https://docs.uniswap.org/contracts/v3/reference/core/interfaces/pool/IUniswapV3PoolEvents#swap

#### sqrtPriceX96：
The sqrt(price) of the pool after the swap, as a Q64.96

存储的方式是X96，所以得处理一下，除以2**96，得到sqrt p，然后平方一下得到p

$$ sqrtPriceX96  = \sqrt{p} \times 2 ^ {96}$$

#### liquidity：
The liquidity of the pool after the swap
#### tick：
The log base 1.0001 of price of the pool after the swap


In [169]:
# convert str into float
swaps['amount0'] = swaps['amount0'].astype(float)
swaps['amount1'] = swaps['amount1'].astype(float)
swaps['amountUSD'] = swaps['amountUSD'].astype(float)
swaps['sqrtPriceX96'] = swaps['sqrtPriceX96'].astype(float)
swaps['tick'] = swaps['tick'].astype(int)
swaps['logIndex'] = swaps['logIndex'].astype(int)
# transform X86 into normal number
swaps["sqrtPriceX96"] = (swaps["sqrtPriceX96"] / (2**96) ) ** 2
# transform sqrt_p into p
swaps["sqrtPriceX96"] = 1 / (swaps["sqrtPriceX96"] * (10 ** -12))


In [139]:
swaps

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex
0,0x000128f2e478320e005d9435b14ad71341d06b8d1d7f...,1679404235,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0xfa7d0cfee4d881cf8159e0d18c692ccb71e9c0c5,0xfa7d0cfee4d881cf8159e0d18c692ccb71e9c0c5,-5000.197259,2.750358,4998.420144,1818.915412,201260,209
1,0x0003911713a80bfe444cae965b8d0670738285e7fcad...,1679572631,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,0x77a879f392cc765064c12707d2f79b63025b7590,-4711.072418,2.675849,4706.184083,1761.460331,201581,154
2,0x0004a39f7ec128b26923260e7c032a86d7d49e412332...,1679339867,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x8aed6a72c227bf812727eddb719730f38fc01519,1102.949604,-0.625765,1104.028921,1761.681703,201580,107
3,0x00050a33136a426b48d4a9259fa3ae414c09e013eb12...,1679274431,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x3765240be64af984bc5bb3b6a2988e431d668f6b,0x3765240be64af984bc5bb3b6a2988e431d668f6b,-49.523861,0.027870,49.488922,1777.848337,201488,357
4,0x0005caec68a2686fe1ff47d81c1d1ed965b9066aa9fa...,1679515811,0x280027dd00ee0050d3f9d168efd6b40090009246,0x280027dd00ee0050d3f9d168efd6b40090009246,0xeeb75213df02f4d3d0c7943a3b76c199bb4b8d99,-385850.858553,223.236593,385516.102879,1728.294361,201771,11
...,...,...,...,...,...,...,...,...,...,...,...
95,0x007b86e6f6efea018ba8cb08c3d8e0bc537b02e0d1fd...,1679171795,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x2493c86b62e8ff26208399144817ef2898c59460,0x2493c86b62e8ff26208399144817ef2898c59460,-1796.800653,1.000000,1797.094373,1797.698546,201377,165
96,0x007c05582bf51c754d2532f2197152d2e765e7338049...,1679324363,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xe66b31678d6c16e9ebf358268a790b763c133750,0xc1136611c10081bbd18e3c3977747d04a1f23d0a,-1651.417392,0.929205,1649.800858,1778.123774,201487,23
97,0x007f5b58257f62d002294a27b5dc03b47a25c5a09c91...,1679246231,0xe8cfad4c75a5e1caf939fd80afcf837dde340a69,0xe8cfad4c75a5e1caf939fd80afcf837dde340a69,0x24f7ef98522dd61d529464f67bb3ffe96ea8afc2,-111049.732062,60.927138,110984.073197,1823.20956,201236,39
98,0x00803a3031c82b3122ff8407af9e633de32519a75607...,1679382947,0x4a5a7331da84d3834c030a9b8d4f3d687a3b788b,0x4a5a7331da84d3834c030a9b8d4f3d687a3b788b,0x7b29c3f7648e825a774a89463a35ad1b5895c490,499.750000,-0.289368,500.249452,1726.17974,201783,370


In [144]:
# 首先不考虑tick被击穿的情况, 表中的tick是本次swap之后池子内的swap，为了先简化计算，假设最终的tick在我们选定的头寸之内的所有swaps都是在我们的头寸之内发生的
my_tick_lower = 198760
my_tick_upper = 204080
# 筛选swaps的条件：swap后最终的tick在我们的头寸tick之内
condition = (swaps['tick'] > my_tick_lower) & (swaps['tick'] < my_tick_upper)
# 筛选
print("total swaps: \n", swaps.shape[0])
swaps = swaps[condition]
print("total swaps in range: \n", swaps.shape[0])
# 好吧这个人的range选的太太大了，一直in range, 不过这样算手续费就很准

total swaps: 
 100
total swaps in range: 
 100


In [245]:
# 来计算手续费
# 先复制一个swap 用来乱算
swaps1 = swaps
# 先把不用的列去掉
swaps1 = swaps1.drop(swaps1.columns[[0, 2, 3, 4, 7, 10]], axis = 1)
swaps1.head(6)

,timestamp,amount0,amount1,sqrtPriceX96,tick,date
0,1679680919,-19922.526611,11.442382,1741.953774,201692,2023-03-25 02:01:59
1,1679774651,-12033.180287,7.000000,1719.864424,201820,2023-03-26 04:04:11
2,1679404235,-5000.197259,2.750358,1818.915412,201260,2023-03-21 21:10:35
3,1679879915,-226399.210721,127.909247,1770.542895,201529,2023-03-27 09:18:35
4,1679572631,-4711.072418,2.675849,1761.460331,201581,2023-03-23 19:57:11
5,1679339867,1102.949604,-0.625765,1761.681703,201580,2023-03-21 03:17:47


#### position liquidity
计算手续费的时候还有一个重要的点是头寸的流动性，因为手续费是按照头寸流动性占总流动性的比例派发的。

v2中计算你的liquidity是多少非常简单：

$$ L = x \times y $$

在v3中由于virtual reserve的存在，计算liquidity的公式变成（推导参考：https://uniswapv3book.com/docs/milestone_1/calculating-liquidity/#liquidity-amount-calculation和https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf

推导过程：

首先在v3中流动性的式子变成了这样：

$$ (x_{real} + \frac {L} {\sqrt{p_{b}}}) \times (y_{real} +  L \times {\sqrt{p_{a}}}) = L ^2 $$
(pa是头寸的价格下限，pb是上限)

当$ P \le p_{a} $的时候，池子里只有x，y = 0

$$ (x_{real} + \frac {L} {\sqrt{p_{b}}}) \times  L \times {\sqrt{p_{a}}} = L ^2 $$

当$ P \ge p_{b} $的时候，池子里只有y，x = 0

$$ \frac {L} {\sqrt{p_{b}}} \times (y_{real} +  L \times {\sqrt{p_{a}}}) = L ^2 $$

整理上面的两个式子，可以得到L的两个计算公式：

1. 

$$ L = x \times \frac {\sqrt{p_{a}} \cdot \sqrt{p_{b}}} {\sqrt{p_{a}} - \sqrt{p_{b}}} $$

2. 

$$ L = \frac {y} {\sqrt{p_{a}} - \sqrt{p_{b}}} $$

将两个值都计算出来之后，选择小的那一个。

### 计算tick range内的总流动性

In [261]:
# $$ sqrtPriceX96  = \sqrt{p} \times 2 ^ {96}$$
p_lower = (1.0001 ** 198760) 
p_upper = (1.0001 ** 204080) 

sqrt_p_lower = (p_lower ** 0.5) * (2 ** 96)
sqrt_p_upper = (p_upper ** 0.5) * (2 ** 96)

print('this is sqrt price lower: {:.0f}'.format(sqrt_p_lower))
print('this is sqrt price upper:{:.0f}'.format(sqrt_p_upper))



this is sqrt price lower: 1639390289596216154630867994542080
this is sqrt price upper:2138941539491792332891588793466880


'1.6393902895962162e+33'

In [196]:
# 首先我们已经有了池子liquidity所有tick的流动性数据，需要计算出池子里在我们的头寸范围内的总流动性
# 我本来以为算这个会很麻烦，但是我发现query_liquidity函数返回的数据中，liquidity就是in range的liquidity。
liquidity_data = swapData.query_liquidity(1679068800, 1679673600, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "1639390289596216154630867994542080", "2138941539491792332891588793466880")
liquidity_data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1679072400,30788832203141306590,1906022337482011155206725434245622,1727.838693579332397287823785551691,0.0005787577299408856894168926824109645,201774,2312158785098937052494570152217525,1109665930631382897574108289654189272954102,363741182.8303982633120492904007652,0,0,0,0,349,1715.984479040217612635287842814415,1734.247814416736937143046816128599,1715.719586209013996779322671793541,1727.838693579332397287823785551691
1,1679076000,30335451111637185983,1902228344319510065920412019047373,1734.737913210422806755592553486249,0.0005764559547495751999689070796788115,201734,2312216559855034939422879201028187,1109691771721878689625682223884054944269262,364112711.2526927733521560182183211,0,0,0,0,317,1727.836976511951527664991922547199,1734.737913210422806755592553486249,1717.648119913004161973194737873801,1734.737913210422806755592553486249
2,1679079600,29136272383745257041,1896207166369707888540506256730392,1745.772308123954713225775051712259,0.0005728123852958934694297346816470754,201670,2312311586355258922806996249821275,1109733988873390402439915596012262413890369,361535632.0601155211202920489855564,0,0,0,0,334,1734.712030717442333568942707632399,1746.684565418445477867887983012907,1732.851071987621437218834629969072,1745.772308123954713225775051712259
3,1679083200,29155526732098504706,1897039698024161669848304803585763,1744.240350950043333371334764406051,0.0005733154834167925578956548047763606,201679,2312405216177388689787126610454526,1109788781038653813021677282971350131285545,364444641.3551118523700690778229295,0,0,0,0,307,1746.751187500000067076289182830219,1756.889383965950215989609532840648,1733.118336656699146001761654660613,1744.240350950043333371334764406051
4,1679086800,27547634722926741243,1887391159854370169541208055025004,1762.119407182205743461631236756587,0.0005674984316750099366809510866950609,201577,2312562964065321587002564097838970,1109857533797043930382095487494978532388488,365361117.7910602923638770894625618,0,0,0,0,263,1744.240422680368809736053196035087,1771.923466943892387594257067879091,1744.238112317283881456801956157836,1762.119407182205743461631236756587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1679414400,22665294004126195183,1857884871127377029977660379809271,1818.534610389971169936976386506151,0.0005498933010604386935725984037351437,201262,2320399686333570447276910166402010,1114166617967978477504068631093241490648624,366942831.3671984653858230310347039,0,0,0,0,357,1819.278319413313248121674387334605,1830.959978202350362267645956765388,1815.186468059222795879963663121857,1818.534610389971169936976386506151
96,1679418000,17106813549992212750,1868686743100013891679632148956590,1797.571435435975779714669736576407,0.000556306125190214765128457575122897,201378,2320502663308170312210723325621296,1114246834487638545052400087381527734350518,365423483.6197843767903618664969388,0,0,0,0,351,1818.436905060612240757071491293712,1819.939401548029980479729114733543,1795.298631312076007853225739674567,1797.571435435975779714669736576407
97,1679421600,17038251628284370262,1870429810153332958514408885963420,1794.222656923737406500509468521021,0.0005573444277615676834770828566827302,201397,2320641718241644453239428098165667,1114328287237178879374137489174114521353232,364416217.6858984541887904130509127,0,0,0,0,334,1797.535614951426977460695257089418,1797.668357493559105968057010735421,1781.244282781489017545848534911882,1794.222656923737406500509468521021
98,1679425200,17122891510493817607,1869430120490411616981522659064539,1796.142113993094772825548029797743,0.000556748818598128186348804594208217,201386,2320701919714646456584563588926019,1114360681060763779372256155664474260509884,365369358.0926862050578731684620782,0,0,0,0,295,1794.22250935879446761969026422345,1804.302613762839697654901698418324,1

In [258]:
my_ratio = 286206700622530/30788832203141306590 # 暂时随便选了个数算了个大概

swaps1["fee0"] = abs(swaps1["amount0"] * my_ratio) * 0.0005
fee0_sum = swaps1["fee0"].sum()

swaps1["fee1"] = abs(swaps1["amount1"] * my_ratio) * 0.0005
fee1_sum = swaps1["fee1"].sum()

print(fee0_sum)
print(fee1_sum)
print('{:.0f}'.format(fee0_sum))
print('{:.0f}'.format(fee1_sum))
# 对比这个比例和revert.finance上的，相差不大，算的应该是对的，但是数值对不上，我发现可能是swaps数据没有拿全，用query_swaps拿到的每次都只有100条
# 解决： 这是graph ql的特征，每次只返回100行。需要用分页查询来查询所有的数据

0.014280091727154541
8.06797991361679e-06
0
0


In [270]:
print('this is sqrt price lower: {:.0f}'.format(sqrt_p_lower))
print('this is sqrt price upper:{:.0f}'.format(sqrt_p_upper))

sqrt_p_lower_f = '{:.0f}'.format(sqrt_p_lower)
sqrt_p_upper_f = '{:.0f}'.format(sqrt_p_upper)

this is sqrt price lower: 1639390289596216154630867994542080
this is sqrt price upper:2138941539491792332891588793466880


In [361]:
begin = datetime(2023, 3, 28, 2, 16, 29, 759856)
end = datetime(2023, 3, 28, 12, 0, 29, 759856)
data = swapData.query_swaps(int(begin.timestamp()), int(end.timestamp()), pool_id)

querying... The current page is:  0
querying... The current page is:  1000
querying... The current page is:  2000
querying... The current page is:  3000
querying... The current page is:  4000
querying... The current page is:  5000
querying... The current page is:  6000
querying... The current page is:  7000
querying... The current page is:  8000
querying... The current page is:  9000
querying... The current page is:  10000
querying... The current page is:  11000
querying... The current page is:  12000
querying... The current page is:  13000
querying... The current page is:  14000
querying... The current page is:  15000
querying... The current page is:  16000
querying... The current page is:  17000
querying... The current page is:  18000
querying... The current page is:  19000
querying... The current page is:  20000
querying... The current page is:  21000
querying... The current page is:  22000
querying... The current page is:  23000
querying... The current page is:  24000
querying... T